In [2]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import random
import time
import numpy as np
import cv2
from PIL import Image, ImageEnhance

# --- Google Drive Path Configuration ---
source_folder = r"D:\Assistant Professoor Masum Billal Sir\4 Dataset for Leaf Disease(Masum Billah Sir)\Agriculture Combined Dataset\Papaya Curled Yellow Spot"
target_folder = r"C:\Users\Sagor\Downloads\Output Papaya Curled Yellow Spot"
target_images = 1000  # desired total number of images

# --- Ensure Target Folder Exists ---
os.makedirs(target_folder, exist_ok=True)

# --- PIL-Based Augmentation (Color / Contrast / Brightness) ---
def augment_image_pil(image, count):
    brightness_factor = random.uniform(0.7, 1.3)
    contrast_factor = random.uniform(0.7, 1.3)
    color_factor = random.uniform(0.7, 1.3)

    enhanced_image = ImageEnhance.Color(
        ImageEnhance.Contrast(
            ImageEnhance.Brightness(image).enhance(brightness_factor)
        ).enhance(contrast_factor)
    ).enhance(color_factor)

    filename = f"aug_pil_{count}_b{brightness_factor:.2f}_c{contrast_factor:.2f}_col{color_factor:.2f}.jpg"
    return enhanced_image, filename


# --- OpenCV-Based Augmentation (Flipping, Rotation, Zooming, Noise, Shifting, Brightness) ---
def augment_image_cv2(image_path, count):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")

    height, width = image.shape[:2]
    augmented_images = []

    # === 1️⃣ Flipping ===
    augmented_images.append((cv2.flip(image, 1), f"cv2_flip_h_{count}.jpg"))  # Horizontal
    augmented_images.append((cv2.flip(image, 0), f"cv2_flip_v_{count}.jpg"))  # Vertical

    # === 2️⃣ Rotation ===
    for angle in [15, -15, 30, -30]:
        M = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
        rotated = cv2.warpAffine(image, M, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
        filename = f"cv2_rotate_{angle}_{count}.jpg"
        augmented_images.append((rotated, filename))

    # === 3️⃣ Zooming ===
    for zoom_factor in [0.8, 1.0, 1.2]:
        if zoom_factor == 1.0:
            zoomed = image.copy()
        elif zoom_factor > 1.0:
            zh, zw = int(height * zoom_factor), int(width * zoom_factor)
            zoomed_large = cv2.resize(image, (zw, zh), interpolation=cv2.INTER_LINEAR)
            start_x, start_y = (zw - width) // 2, (zh - height) // 2
            zoomed = zoomed_large[start_y:start_y + height, start_x:start_x + width]
        else:
            zh, zw = int(height * zoom_factor), int(width * zoom_factor)
            zoomed_small = cv2.resize(image, (zw, zh), interpolation=cv2.INTER_LINEAR)
            zoomed = np.zeros_like(image)
            start_x, start_y = (width - zw) // 2, (height - zh) // 2
            zoomed[start_y:start_y + zh, start_x:start_x + zw] = zoomed_small

        filename = f"cv2_zoom_{zoom_factor}_{count}.jpg"
        augmented_images.append((zoomed, filename))

    # === 4️⃣ Adding Gaussian Noise ===
    mean, stddev = 0, 10
    noise = np.random.normal(mean, stddev, image.shape).astype(np.float32)
    noisy_image = image.astype(np.float32) + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    filename = f"cv2_noisy_{count}.jpg"
    augmented_images.append((noisy_image, filename))

    # === 5️⃣ Shifting ===
    for dx, dy in [(10, 0), (-10, 0), (0, 10), (0, -10)]:
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        shifted = cv2.warpAffine(image, M, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
        filename = f"cv2_shift_{dx}_{dy}_{count}.jpg"
        augmented_images.append((shifted, filename))

    # === 6️⃣ Brightness Adjustment ===
    for brightness_factor in [0.7, 1.3]:
        bright_img = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)
        filename = f"cv2_brightness_{brightness_factor}_{count}.jpg"
        augmented_images.append((bright_img, filename))

    return augmented_images


# --- Manage Target Folder Limit ---
def manage_target_folder(folder, max_images):
    images = sorted(
        [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
        key=lambda x: os.path.getctime(os.path.join(folder, x))
    )
    if len(images) > max_images:
        for img in images[max_images:]:
            os.remove(os.path.join(folder, img))


# --- Main Augmentation Pipeline ---
def main():
    print("🔍 Checking source folder...")
    if not os.path.isdir(source_folder):
        print(f"❌ Source folder not found: {source_folder}")
        return
    else:
        print(f"✅ Source folder found: {source_folder}")

    # List image files
    files_in_source = os.listdir(source_folder)
    print(f"📂 Total files found: {len(files_in_source)}")
    if files_in_source:
        print("🔸 Sample files:", files_in_source[:5])

    image_paths = [
        os.path.join(source_folder, f)
        for f in files_in_source
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]

    if not image_paths:
        print("⚠️ No valid image files found (.jpg/.jpeg/.png).")
        return
    else:
        print(f"✅ Found {len(image_paths)} valid image(s) for augmentation.")

    random.shuffle(image_paths)

    start_time = time.time()
    current_count = len([f for f in os.listdir(target_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"🚀 Starting augmentation. Already in target: {current_count} images.")

    while current_count < target_images:
        for img_path in image_paths:
            if current_count >= target_images:
                break
            try:
                # --- PIL Augmentation ---
                with Image.open(img_path) as pil_image:
                    if pil_image.mode != "RGB":
                        pil_image = pil_image.convert("RGB")

                    pil_aug, pil_filename = augment_image_pil(pil_image, current_count)
                    pil_aug.save(os.path.join(target_folder, pil_filename), format="JPEG")
                    print(f"✅ Saved PIL image: {pil_filename}")
                    current_count += 1

                # --- OpenCV Augmentations ---
                cv2_augments = augment_image_cv2(img_path, current_count)
                for aug_img, filename in cv2_augments:
                    if current_count >= target_images:
                        break
                    cv2.imwrite(os.path.join(target_folder, filename), aug_img)
                    print(f"✅ Saved CV2 image: {filename}")
                    current_count += 1

            except Exception as e:
                print(f"⚠️ Error processing {img_path}: {e}")

    # --- Limit total images if exceeded ---
    manage_target_folder(target_folder, target_images)
    total = len(os.listdir(target_folder))
    print(f"\n✅ Augmentation complete. Total images: {total}")
    print(f"⏱ Time taken: {time.time() - start_time:.2f} seconds")


# --- Run Script ---
if __name__ == "__main__":
    main()


🔍 Checking source folder...
✅ Source folder found: D:\Assistant Professoor Masum Billal Sir\4 Dataset for Leaf Disease(Masum Billah Sir)\Agriculture Combined Dataset\Papaya Curled Yellow Spot
📂 Total files found: 86
🔸 Sample files: ['IMG-20241027-WA0027-removebg-preview.png', 'IMG-20241027-WA0038-removebg-preview.png', 'IMG-20241027-WA0046-removebg-preview.png', 'IMG-20241027-WA0057-removebg-preview.png', 'IMG-20241027-WA0058-removebg-preview.png']
✅ Found 86 valid image(s) for augmentation.
🚀 Starting augmentation. Already in target: 0 images.
✅ Saved PIL image: aug_pil_0_b1.07_c0.74_col0.97.jpg
✅ Saved CV2 image: cv2_flip_h_1.jpg
✅ Saved CV2 image: cv2_flip_v_1.jpg
✅ Saved CV2 image: cv2_rotate_15_1.jpg
✅ Saved CV2 image: cv2_rotate_-15_1.jpg
✅ Saved CV2 image: cv2_rotate_30_1.jpg
✅ Saved CV2 image: cv2_rotate_-30_1.jpg
✅ Saved CV2 image: cv2_zoom_0.8_1.jpg
✅ Saved CV2 image: cv2_zoom_1.0_1.jpg
✅ Saved CV2 image: cv2_zoom_1.2_1.jpg
✅ Saved CV2 image: cv2_noisy_1.jpg
✅ Saved CV2 image